In [1]:
from random import random, seed
from itertools import product
import numpy as np
from matplotlib import pyplot as plt
from icecream import ic
from tqdm import tqdm
import pandas as pd

In [3]:
pd.read_csv("./cities/china.csv")


,Acheng,45.54,126.96000000000001
0,Aksu,41.15,80.250000
1,Alaer,40.515556,81.263611
2,Altay,47.84,88.130000
3,Anbu,23.46,116.680000
4,Anda,46.4,125.320000
...,...,...,...
720,Ziyang,30.14,104.640000
721,Zoucheng,35.4,116.966667
722,Zouxian,35.410000000000004,116.940000
723,Zunhua,40.183333,117.966667
